In [ ]:
!python -m pip install --user -r ../requirements.txt

In [ ]:
!curl "https://echanges.dila.gouv.fr/OPENDATA/ACCO/" | grep "<a" | grep "tar.gz" | cut -d">" -f3 | cut -d"<" -f1 | grep "ACCO" | sed -e "s/^/https:\/\/echanges.dila.gouv.fr\/OPENDATA\/ACCO\//g" | xargs -n 1 curl -O 

In [ ]:
!ls  | grep "ACCO"   | xargs -n 1 tar -xzvf 

In [ ]:
!curl "https://echanges.dila.gouv.fr/OPENDATA/ACCO/Freemium_acco_global_20220417-170000.tar.gz" -o "accords.tar.gz"

In [ ]:
!tar -xzvf "accords.tar.gz"

In [ ]:
!mkdir accords 

In [ ]:
!find acco -name '*.docx' -exec mv {} accords \;  

In [ ]:
import pandas as pd
import docx
import os

In [ ]:
DIR_DOCX = "./accords/"
OUTPUT_FILE_PARQUET = "my_file.parquet"

In [ ]:
def read_txt_from_docx(filename: str) -> str:
    fullText = []
    try:
        doc = docx.Document(filename)
        for para in doc.paragraphs:
            for run in para.runs:
                if run.font.bold:
                    print(run.text)
                    #fullText.append(para.text)

    except:
        print(f"bug docx {filename}")
    return "\n".join(fullText)

In [ ]:
read_txt_from_docx(DIR_DOCX+"T06320002150-45322362000015.docx")